# Partitioned Custom ML Model with Model Registry

Many datasets can be easily partitioned into multiple independent subsets. For example, a dataset containing sales data for a chain of stores can be partitioned by store number. A separate model can then be trained for each partition. Training and inference operations on the partitions can be parallelized, reducing the wall-clock time for these operations. Furthermore, since individual stores likely differ somewhat in how their features affect their sales, this approach can actually lead to more accurate inference at the store level.

With the Snowflake Model Registry, you implement partitioned training and inference using custom models. When using the model, the registry partitions the dataset, fits and predicts the partitions in parallel using all the nodes and cores in your warehouse, and combines the results into a single dataset afterward.

In this notebook we will complete the following steps:
1. Load Partitioned_Custom_Model_Restaurant_Traffic_Data.csv from a stage. This data set contains the hourly traffic at restaurants
2. Define an XGBoost model to predict hourly traffic
3. Log this model to the model registry
4. Use Partitioned Custom Models to run training and inference for each store in the data

In [1]:
from datetime import timedelta

import pandas as pd

from snowflake.ml.model import custom_model
from snowflake.ml.model import model_signature
from snowflake.ml.registry import registry
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F

session = get_active_session()

# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit", "name":"notebook_pack_partitioned_ml", "version":{"major":1, "minor":0}}

# Set session context 
session.use_role(f"{{ env.DATAOPS_CATALOG_SOLUTION_PREFIX }}_DATA_SCIENTIST") 

# Print the current role, warehouse, and database/schema
print(f"role: {session.get_current_role()} | WH: {session.get_current_warehouse()} | DB.SCHEMA: {session.get_fully_qualified_current_schema()}")

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


[Row(status='Statement executed successfully.')]

## Load and Preview Data

The dataset is loaded locally from the `Partitioned_Custom_Model_Restaurant_Traffic_Data.csv` file. It contains an epoch timestamp in milliseconds, a store ID which will later be used as a partition column, a feature column `COLLEGE_TOWN`, and a target to be forecasted, `HOURLY_TRAFFIC`.

In [ ]:
status = session.file.get("@{{ env.DATAOPS_DATABASE }}.analytics.partitioned_data/Partitioned_Custom_Model_Restaurant_Traffic_Data.csv","./")
status[0][2]

In [ ]:
test_df_pandas = pd.read_csv("Partitioned_Custom_Model_Restaurant_Traffic_Data.csv")
test_df = session.create_dataframe(test_df_pandas)
test_df

## Forecasting Model
We'll now define our XGBoost forecasting model. This is a [Custom Model](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/custom-models) and will be used for partitioned training and inference later.  

In [5]:
class ForecastingModel(custom_model.CustomModel):

    # Use the same decorator as for methods with FUNCTION inference.
    @custom_model.partitioned_inference_api
    def predict(self, df: pd.DataFrame) -> pd.DataFrame:        
        import xgboost

        # Set the time column as our index.
        input_df = df.set_index('EPOCH')
        input_df.index = pd.to_datetime(df['EPOCH'], unit='ms')

        # Generate categorical features using the datetime index.
        input_df['HOUR'] = input_df.index.hour.astype("category")
        input_df['DAY_OF_WEEK'] = input_df.index.dayofweek.astype("category")
        input_df['MONTH'] = input_df.index.month.astype("category")
        input_df['YEAR'] = input_df.index.year.astype("category")
        
        input_df['COLLEGE_TOWN'] = input_df['COLLEGE_TOWN'].astype("category")
        
        # Use get_dummies (one-hot encoding) for categorical features.
        final = pd.get_dummies(data=input_df, columns=['COLLEGE_TOWN', 'HOUR', 'MONTH', 'YEAR', 'DAY_OF_WEEK'])

        # Define the train & forecast split thresholds.
        today = pd.to_datetime('2022-10-01')
        yesterday = today - timedelta(days=1)
        four_weeks = today + timedelta(days=28)
        tomorrow = today + timedelta(days=1)

        # Train data starts on June 16th 2018 and ends on September 30th.
        train = final[(final.index >= pd.to_datetime('16-Jun-2018')) & (final.index <= pd.to_datetime(yesterday))]
        
        # The forecast starts from October 1st 2022 and goes 4 weeks into the future.
        forecast = final[(final.index >= pd.to_datetime(tomorrow)) & (final.index <= pd.to_datetime(four_weeks))].copy()

        # Remove the target from the input dataset, and construct target dataset.
        X_train = train.drop('HOURLY_TRAFFIC', axis=1)
        y_train = train['HOURLY_TRAFFIC']

        X_forecast = forecast.drop('HOURLY_TRAFFIC', axis=1)
        
        # Train an XGBoost regression model.
        model = xgboost.XGBRegressor(n_estimators=200, n_jobs=1)
        model.fit(X_train, y_train, verbose=False)

        # Predict the hourly forecast for the future dates and make sure no predictions are less than zero.
        forecast.loc[:, 'PREDICTION'] = model.predict(X_forecast)
        forecast.loc[:, 'EPOCH_OUT'] = [t.value // 10**9 for t in forecast.index]
        forecast = forecast[['EPOCH_OUT', 'PREDICTION']]
        forecast = forecast.sort_index()
        forecast.loc[forecast['PREDICTION'] < 0, 'PREDICTION'] = 0

        return forecast

In [6]:
my_forecasting_model = ForecastingModel()

### Predict Locally
The predict method can be tested locally by using a pandas dataframe directly. Here we can run `predict` for a single partition.

In [7]:
my_forecasting_model.predict(test_df_pandas.loc[test_df_pandas['STORE_ID'] == 1])

/var/folders/0l/n0lxdw_j2xq9f0l8qq88zt1h0000gn/T/ipykernel_30436/948395715.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['PREDICTION'] = model.predict(X_forecast)
/var/folders/0l/n0lxdw_j2xq9f0l8qq88zt1h0000gn/T/ipykernel_30436/948395715.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['EPOCH_OUT'] = [t.value // 10**9 for t in forecast.index]


,EPOCH_OUT,PREDICTION
EPOCH,,
2022-10-02 07:00:00,1664694000,77.787636
2022-10-02 08:00:00,1664697600,76.995056
2022-10-02 09:00:00,1664701200,76.751877
2022-10-02 10:00:00,1664704800,76.600456
2022-10-02 11:00:00,1664708400,96.709358
...,...,...
2022-10-28 18:00:00,1666980000,30.727842
2022-10-28 19:00:00,1666983600,30.608843
2022-10-28 20:00:00,1666987200,31.198336


## Model Registry
We will start by creating a [Snowflake Model Registry](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/overview) in our database and schema.

In [2]:
REGISTRY_DATABASE_NAME = "{{ env.DATAOPS_DATABASE }}"
REGISTRY_SCHEMA_NAME = "ANALYTICS"

reg = registry.Registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

### Log the model
We will log the model to our model registry, specifying the `function_type: "TABLE_FUNCTION"` option.

In [9]:
options = {
    "function_type": "TABLE_FUNCTION",
    "relax_version": True,
}

mv = reg.log_model(
    my_forecasting_model,
    model_name="forecast",
    conda_dependencies=["pandas", "scikit-learn", "xgboost"],
    options=options,
    signatures={
        "predict": model_signature.ModelSignature(
            inputs=[
                model_signature.FeatureSpec(name="EPOCH", dtype=model_signature.DataType.DOUBLE),
                model_signature.FeatureSpec(name="STORE_ID", dtype=model_signature.DataType.DOUBLE),
                model_signature.FeatureSpec(name="COLLEGE_TOWN", dtype=model_signature.DataType.DOUBLE),
                model_signature.FeatureSpec(name="HOURLY_TRAFFIC", dtype=model_signature.DataType.INT64),
            ],
            outputs=[
                model_signature.FeatureSpec(name="EPOCH_OUT", dtype=model_signature.DataType.FLOAT),
                model_signature.FeatureSpec(name="PREDICTION", dtype=model_signature.DataType.FLOAT),
            ],
        )
    },
)

/Users/thoyt/anaconda3/envs/snowpark-ml-dataiku/lib/python3.11/contextlib.py:137: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)


## Run Partitioned Model
Finally, we will run the model from our model registry. We can use the `run` method for inference, specifying the partition column.

In [10]:
result = mv.run(test_df, partition_column="STORE_ID")
result.select("EPOCH_OUT", "PREDICTION", "STORE_ID").to_pandas()

,EPOCH_OUT,PREDICTION,STORE_ID
0,1.664694e+09,77.972305,40.0
1,1.664698e+09,77.546402,40.0
2,1.664701e+09,76.611122,40.0
3,1.664705e+09,76.296654,40.0
4,1.664708e+09,98.838348,40.0
...,...,...,...
86395,1.666980e+09,31.088211,78.0
86396,1.666984e+09,30.569330,78.0
86397,1.666987e+09,32.221611,78.0
86398,1.666991e+09,6.004010,78.0
